# Clifford Sampling

Sampling random n-qudit Clifford circuits is at the core of the most popular benchmarking technique for quantum computers: Randomized Benchmarking.

In [ ]:
# Uncomment and run if the package has not been installed in the working environment.
# %pip install -e .
from qdit.benchmarking import Tableau
from qdit.benchmarking import sample_clifford

Qdit supports generating random clifford via the `Tableau` class:

In [4]:
n_qudits = 2
dimension = 2

t = Tableau(n_qudits, dimension, entangling_gate='iSWAP')
t.populate_and_sweep(display=True) #set display to True to see Tableau steps
print(t.circuit)

0 (d=2): ───────

1 (d=2): ───────
0|1 	 1|0 	 0
1|1 	 1|0 	 0


0|0 	 0|0 	 0
0|0 	 0|0 	 0


Sweeping...
Step 1:
0 (d=2): ───────F───

1 (d=2): ───────────
1|1 	 0|0 	 0
1|1 	 1|0 	 0


0|0 	 0|0 	 0
0|0 	 0|0 	 0


Step 2:
0 (d=2): ───────F───C───C───⨂───
                            │
1 (d=2): ───────C───────────⨂───
1|0 	 0|0 	 0
1|1 	 1|0 	 0


0|0 	 0|0 	 0
0|0 	 0|0 	 0


Step 3: skipped
Step 4: Repeat for second row
0 (d=2): ───────F───C───C───⨂───
                            │
1 (d=2): ───────C───────────⨂───
1|0 	 0|0 	 0
1|1 	 1|0 	 0


0|0 	 0|0 	 0
0|0 	 0|0 	 0


0 (d=2): ───────F───C───C───⨂───F───
                            │
1 (d=2): ───────C───────────⨂───────
0|0 	 1|0 	 0
1|1 	 1|0 	 0


0|0 	 0|0 	 0
0|0 	 0|0 	 0


0 (d=2): ───────F───C───C───⨂───F───S───
                            │
1 (d=2): ───────C───────────⨂───────────
0|0 	 1|0 	 0
1|1 	 0|0 	 0


0|0 	 0|0 	 0
0|0 	 0|0 	 0


[0, 1]
[1 1]
0 (d=2): ───────F───C───C───⨂───F───S───F───
                      

In [29]:
import numpy as np

n_qudits = 2
dimension = 2
n_iter = 500

two_qudit_gate_list_iswap = []
for _ in range(n_iter):
    t_iswap = Tableau(n_qudits, dimension, entangling_gate='iSWAP')
    two_qudit_gate_count = 0
    t_iswap.populate_and_sweep(display=False)
    for moment in t_iswap.circuit:
        for op in moment:
            if op._num_qubits_() > 1: two_qudit_gate_count+=1
    two_qudit_gate_list_iswap.append(two_qudit_gate_count)

print(f"Avg 2-qudit gate count for 2-qudit iswap circuit: {np.mean(np.array(two_qudit_gate_list_iswap))}")
print(f"Stdev 2-qudit gate count for 2-qudit iswap circuit: {np.std(np.array(two_qudit_gate_list_iswap))}")


two_qudit_gate_list_cx = []
for _ in range(n_iter):
    t_cx = Tableau(n_qudits, dimension, entangling_gate='CX')
    two_qudit_gate_count = 0
    t_cx.populate_and_sweep(display=False)
    for moment in t_cx.circuit:
        for op in moment:
            if op._num_qubits_() > 1: two_qudit_gate_count+=1
    two_qudit_gate_list_cx.append(two_qudit_gate_count)
print(f"Avg 2-qudit gate count for 2-qudit cx circuit: {np.mean(np.array(two_qudit_gate_list_cx))}")
print(f"Stdev 2-qudit gate count for 2-qudit cx circuit: {np.std(np.array(two_qudit_gate_list_cx))}")


Avg 2-qudit gate count for 2-qudit iswap circuit: 2.468
Stdev 2-qudit gate count for 2-qudit iswap circuit: 1.0435401286007167
Avg 2-qudit gate count for 2-qudit cx circuit: 1.944
Stdev 2-qudit gate count for 2-qudit cx circuit: 1.0415680486650882


These methods are wrapped by the `sample_clifford` function:

In [ ]:
from cirq import unitary
for _ in range(5000):
    c = sample_clifford(n_qudits, dimension)
    assert unitary(c).shape==(4,4), "{}, {}".format(_,c)

We could generate entire n-qudit Clifford groups (if we were naïve and patient). The 11,520-element 2-qubit Clifford group is already stored in this directory. The 2-qutrit Clifford group may take several hours to generate.

In [ ]:
from qdit.benchmarking import Tableau
from cirq import MatrixGate, to_json
import itertools
import numpy as np
n=2
d=2
t = Tableau(n, d)
settings = t.enumerate_settings()
all_pairs = [s for s in itertools.product(*settings)]
print(len(all_pairs))

circ_gen = t.generate_clifford_set()

clifford_group = [MatrixGate(matrix=circ.unitary(), qid_shape=(d,)*int(np.emath.logn(d, len(circ.unitary()[0])))) for circ in circ_gen]

to_json(clifford_group, f"{n}_{d}d_clifford_group.json")